In [ ]:
import os
import Cell_BLAST as cb
import utils
os.environ["CUDA_VISIBLE_DEVICES"] = utils.pick_gpu_lowest_memory()
cb.config.RANDOM_SEED = 0
cb.config.N_JOBS = 4
fixed_model_kwargs = dict(
    latent_dim=10, cat_dim=20,
    epoch=500, patience=20
)

In [ ]:
cb.__version__

---
# Planaria
## Fincher

In [ ]:
fincher = cb.data.ExprDataSet.read_dataset("../../Datasets/data/Fincher/data.h5")
utils.peek(fincher, "build/planaria/Fincher")
fincher.obs.head()

In [ ]:
fincher.obs["cluster"] = "cluster_" + fincher.obs["cluster"]
fincher.obs.dtypes

In [ ]:
fincher_model = cb.directi.fit_DIRECTi(
    fincher, fincher.uns["seurat_genes"],
    **fixed_model_kwargs
)
fincher.latent = fincher_model.inference(fincher)

In [ ]:
ax = fincher.visualize_latent("cell_ontology_class", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/planaria/Fincher/cell_ontology_class.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = fincher.visualize_latent("cell_type1", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/planaria/Fincher/cell_type1.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = fincher.visualize_latent("cluster", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/planaria/Fincher/cluster.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
ax = fincher.visualize_latent("FACS_gate", scatter_kws=dict(rasterized=True))
ax.get_figure().savefig("build/planaria/Fincher/FACS_gate.svg", dpi=utils.DPI, bbox_inches="tight")

In [ ]:
fincher.write_dataset("build/planaria/Fincher/Fincher.h5")

In [ ]:
%%capture capio
fincher_models = [fincher_model]
for i in range(1, cb.config.N_JOBS):
    print("==== Model %d ====" % i)
    fincher_models.append(cb.directi.fit_DIRECTi(
        fincher, fincher.uns["seurat_genes"],
        **fixed_model_kwargs, 
        random_seed=i
    ))
fincher_blast = cb.blast.BLAST(fincher_models, fincher)
fincher_blast.save("build/planaria/Fincher")

In [ ]:
with open("build/planaria/Fincher/stdout.txt", "w") as f:
    f.write(capio.stdout)
with open("build/planaria/Fincher/stderr.txt", "w") as f:
    f.write(capio.stderr)

In [ ]:
utils.self_projection(fincher_blast, "build/planaria/Fincher")

In [ ]:
%%writefile build/planaria/Fincher/predictable.txt
cell_ontology_class
cell_type1
cluster